In [2]:
import numpy as np
from matplotlib import pyplot as plt
import os
import random
from sklearn.model_selection import train_test_split
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


***angry***=0

***disgust***=1

***fear***=2

***happy***=3

***nutral***=4

***sad***=5

***surprised***=6

In [3]:
data_path='E:\\My Projects\\Facial_immotion_Detection\\data\\train'

In [4]:
# img1=cv2.imread(data_path+'\\1\\Training_680349.jpg')

In [5]:
# type(img1)

In [6]:
# img1.shape

In [7]:
# plt.imshow(img1)

In [8]:
classes=["0","1","2","3","4","5","6"]

In [9]:
#resize the image
def resize_img(img_array):
    new_array=cv2.resize(img_array, (224,224))
    # plt.imshow(new_array)
    # plt.show()
    return new_array

***reading all the images and converting them to array***

In [10]:
training_data=[]
def create_training_data():
    for cls in classes:
        path=os.path.join(data_path, cls)
        class_num=classes.index(cls)
        for img in os.listdir(path):
            try:
                img_array= cv2.imread(os.path.join(path, img))
                new_array=resize_img(img_array)
                training_data.append([new_array,class_num])
            except Exception as e:
                pass       
          

In [11]:
create_training_data()

In [12]:
len(training_data)

28709

In [13]:
# shuffle the images
random.shuffle(training_data)

In [14]:
X=[]
y=[]

img_size=244
for feature, label in training_data:
    X.append(feature)
    y.append(label)

X=np.array(X)     #244*244 is the size of image

In [15]:
X.shape

(28709, 224, 224, 3)

***normalize the data***

In [16]:
X=X/255.0           # 255.00 is the value of pure black

In [17]:
X.shape

(28709, 224, 224, 3)

In [18]:
Y=np.array(y)

In [19]:
Y


array([4, 2, 3, ..., 6, 6, 3])

In [20]:
# X_train,Y_train,X_test,Y_test=train_test_split(X,Y,test_size=0.50, shuffle=True)

***training the Deep Learning model***

In [21]:
#model pre-trainning


model=tf.keras.applications.MobileNetV2()
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [22]:
base_input=model.layers[0].input
base_output=model.layers[-2].output

In [23]:
base_output

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [24]:
final_output=layers.Dense(128)(base_output)
final_output=layers.Activation('relu')(final_output)
final_output=layers.Dense(64)(final_output)
final_output=layers.Activation('relu')(final_output)
final_output=layers.Dense(7, activation='softmax')(final_output) # 7 is for 7 different classes


In [25]:
final_output

<KerasTensor: shape=(None, 7) dtype=float32 (created by layer 'dense_2')>

In [26]:
new_model=keras.Model(inputs=base_input, outputs=final_output)

In [27]:
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [28]:
new_model.compile(loss = 'sparse_categorical_crossentropy',  optimizer = 'adam', metrics = ["accuracy"])

In [38]:
new_model.fit(X,Y, epochs = 15)

Epoch 1/15
898/898 [==============================] - 1329s 1s/step - loss: 1.2493 - accuracy: 0.5236
Epoch 2/15
898/898 [==============================] - 1293s 1s/step - loss: 1.0795 - accuracy: 0.5956
Epoch 3/15
898/898 [==============================] - 1289s 1s/step - loss: 0.9979 - accuracy: 0.6288
Epoch 4/15
898/898 [==============================] - 1291s 1s/step - loss: 0.9395 - accuracy: 0.6495
Epoch 5/15
898/898 [==============================] - 1298s 1s/step - loss: 0.8906 - accuracy: 0.6705
Epoch 6/15
898/898 [==============================] - 1298s 1s/step - loss: 0.8423 - accuracy: 0.6877
Epoch 7/15
898/898 [==============================] - 1295s 1s/step - loss: 0.7906 - accuracy: 0.7095
Epoch 8/15
898/898 [==============================] - 1294s 1s/step - loss: 0.7358 - accuracy: 0.7300
Epoch 9/15
898/898 [==============================] - 1295s 1s/step - loss: 0.6810 - accuracy: 0.7508
Epoch 10/15
898/898 [==============================] - 1297s 1s/step - loss: 0.624

In [39]:
new_model.save('final_model_002.h5')

***model loading***

In [40]:
my_model=tf.keras.models.load_model('final_model_002.h5')

***model testing***

In [268]:
test_img=cv2.imread('E:\\My Projects\\Facial_immotion_Detection\\data\\test\\6\\PublicTest_43692311.jpg')

In [269]:
final_image=cv2.resize(test_img, (224,224))
final_image=np.expand_dims(final_image, axis=0)
final_image=final_image/255.0

In [270]:
# final_image.shape

***angry***=0

***disgust***=1

***fear***=2

***happy***=3

***nutral***=4

***sad***=5

***surprised***=6

In [271]:
result=my_model.predict(final_image)

1/1 [==============================] - 0s 25ms/step


In [272]:
np.argmax(result)

6